In [1]:
!pip install opendatasets

In [1]:
import opendatasets as od
from tensorflow import keras
import numpy as np

In [3]:
od.download("https://www.kaggle.com/prashant268/chest-xray-covid19-pneumonia")
# od.download("https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: jessicakaechele
Your Kaggle Key: ··········


100%|██████████| 2.06G/2.06G [00:23<00:00, 95.5MB/s]


In [2]:
LEARNING_RATE = 0.0001
EPOCHS = 2
TARGET_FOLDER = "weights"

In [17]:
import torch
from torchvision import datasets, transforms

Resize Images to 244, 244. By using to_tensor the images are already normalized between 0 and 1. "The image object is an array of (244, 244, 3) should be flattened to be list (178, 608)." What? wie?

In [5]:
transform = transforms.Compose([transforms.Resize((244, 244)),
                                 transforms.ToTensor()])

test_set = datasets.ImageFolder('chest-xray-covid19-pneumonia/Data/test', transform=transform)
train_set = dataset = datasets.ImageFolder('chest-xray-covid19-pneumonia/Data/train', transform=transform)

In [6]:
train_set.targets = torch.tensor(train_set.targets)
train_set.targets[train_set.targets > 0] = 1

test_set.targets = torch.tensor(test_set.targets)
test_set.targets[test_set.targets > 0] = 1

The dataset is extremely unbalanced. Maybe sth which should be handled?  

Creating Sample Data Dictionary
After flattening the data dictionary instance was created for each image sample to represent the image data (features) and its label.
Creating Samples and labels Tensors keras Objects
To build keras the dataset, the keras tensor object should be built for features and keras tensor object for labels.
Create Keras Tensor Dataset
Create keras dataset by using from_tensor_slices API.

nicht nötig?


Data Repetition
Data repeated to simulate the number of clients.
Data Shuffling
Data shuffled to avoid obtaining the same results.
Data Batching
Data grouped into batches to enhance their performance.

vielleicht nicht nötig mit syft? 

In [11]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=True)

Data Mapping
ndarray dataset flattend to 1 darray dataset.
Data Prefetching
Data cached in memory for better performance.

wie?

In Paper aus References haben sie unter anderen ResNet18 benutzt: https://arxiv.org/pdf/2007.05592.pdf

Deshalb würde ich das probieren.

In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Current device: {device}")

Current device: cpu


In [20]:
import torchvision
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 1)

model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [25]:

def transform_labels(labels):
  labels[labels>0] = 1
  return labels

In [15]:
def train(model, data_loader, optimizer):
    """
    model -- neural net
    data_loader -- dataloader for train images
    optimizer -- optimizer
    """
    model.train()
    
    criterion = torch.nn.BCELoss().to(device)
    
    for step, [images, labels] in enumerate(data_loader,0):
        images = images.to(device)
        labels = transform_labels(labels.to(device))
                
        optimizer.zero_grad()
        result = torch.nn.functional.softmax(model(images))
      
        loss = criterion(result, labels.unsqueeze(1).type(torch.FloatTensor))
                
        # backpropagation
        loss.backward()
        optimizer.step()
                                    
        if step%10 == 0:
            print(f"Step: {step}, loss: {loss}")

In [16]:
def calc_accuracy(result, labels):
    result = torch.round(result)

    correct_results_sum = (result == labels).sum().float()
    acc = correct_results_sum/labels.shape[0]
    acc = torch.round(acc * 100)
    
    return acc


In [19]:
def test(model, test_loader, epoch):
    """    
    model -- neural net 
    test_loader -- dataloader of test images
    epoch -- current epoch
    """
    model.eval()
    loss = 0
    accuracy = 0
    for step, [images, labels] in enumerate(test_loader,0):
        images = images.to(device)
        labels = transform_labels(labels.to(device))
        result = torch.nn.functional.softmax(model(images))
        loss += criterion(result.detach(), labels.detach())
        accuracy += calc_accuracy(result.detach(), labels.detach())
    loss /= step
    accuracy /=  step
  
    print(f"Loss: {loss}, Accuracy: {accuracy}")

In [20]:
# initialize optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

# start training
for epoch in range(EPOCHS):
    train(model, train_loader, optimizer)
    
    # save interim weights
    torch.save(model.state_dict(), f'{TARGET_FOLDER}/epoch_{epoch}.ckpt')

    test(model, test_loader, epoch)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


Step: 0, loss: 9.375
Step: 10, loss: 9.375
Step: 20, loss: 9.375
Step: 30, loss: 6.25
Step: 40, loss: 9.375
Step: 50, loss: 9.375
Step: 60, loss: 18.75
Step: 70, loss: 9.375
Step: 80, loss: 6.25
Step: 90, loss: 12.5
Step: 100, loss: 12.5
Step: 110, loss: 12.5
Step: 120, loss: 0.0
Step: 130, loss: 9.375
Step: 140, loss: 15.625
Step: 150, loss: 3.125
Step: 160, loss: 8.333333015441895


FileNotFoundError: ignored

# Federated
https://blog.openmined.org/upgrade-to-federated-learning-in-10-lines/

Neuste Version (0.3.x) hat kein TorchHook -> Lösung finden?

In [3]:
!pip install syft==0.2.9 

In [8]:
import syft as sy


In [43]:
hook = sy.TorchHook(torch)
nr_of_instances = 2
instances = []
for i in range(nr_of_instances):
  instances.append(sy.VirtualWorker(hook, id=str(i)))

In [10]:
class Arguments():
    def __init__(self):
        self.batch_size = 32
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.0001
        self.seed = 1
        self.log_interval = 10
        self.save_model = False
args = Arguments()

torch.manual_seed(args.seed)
kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}

In [13]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)

federated_train_loader = sy.FederatedDataLoader(train_set.federate(instances),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set,
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [27]:
def train(args, model, train_loader, optims, epoch):
    model.train()

    criterion = torch.nn.BCELoss().to(device)

    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location)
        data, target = data.to(device), transform_labels(target.to(device))

        optimizer = optims.get_optim(data.location.id)
        optimizer.zero_grad()
        
        output = torch.nn.functional.softmax(model(data))

        loss = criterion(output, target.unsqueeze(1).type(torch.FloatTensor))

        loss.backward()
        optimizer.step()

        model.get()
        if batch_idx % args.log_interval == 0:
            loss = loss.get()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [49]:
federated_model = torchvision.models.resnet18(pretrained=False)
federated_model.fc = torch.nn.Linear(512, 1)

from syft.federated.floptimizer import Optims
federated_optimizer = Optims(list(range(nr_of_instances)), optim=torch.optim.Adam(params=federated_model.parameters(),lr=args.lr))

for epoch in range(1, args.epochs + 1):
    train(args, federated_model, federated_train_loader, federated_optimizer, epoch)

    if (args.save_model):
      torch.save(federated_model.state_dict(), f'{TARGET_FOLDER}/epoch_{epoch}.ckpt')

    test(federated_model, test_loader, epoch)



/usr/local/lib/python3.7/dist-packages/syft/frameworks/torch/tensors/interpreters/native.py:414: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  response = command_method(*args_, **kwargs_)


Train Epoch: 1 [0/5152 (0%)]	Loss: 3.453878
Train Epoch: 1 [320/5152 (6%)]	Loss: 3.453878
Train Epoch: 1 [640/5152 (12%)]	Loss: 3.453878
Train Epoch: 1 [960/5152 (19%)]	Loss: 7.771225
Train Epoch: 1 [1280/5152 (25%)]	Loss: 4.317347
Train Epoch: 1 [1600/5152 (31%)]	Loss: 2.590408
Train Epoch: 1 [1920/5152 (37%)]	Loss: 7.771225
Train Epoch: 1 [2240/5152 (43%)]	Loss: 3.453878
Train Epoch: 1 [2560/5152 (50%)]	Loss: -6.907755
Train Epoch: 1 [2880/5152 (56%)]	Loss: 0.000000
Train Epoch: 1 [3200/5152 (62%)]	Loss: 0.000000
Train Epoch: 1 [3520/5152 (68%)]	Loss: 0.000000
Train Epoch: 1 [3840/5152 (75%)]	Loss: 0.000000
Train Epoch: 1 [4160/5152 (81%)]	Loss: 0.000000
Train Epoch: 1 [4480/5152 (87%)]	Loss: 0.000000
Train Epoch: 1 [4800/5152 (93%)]	Loss: 0.000000
Train Epoch: 1 [5120/5152 (99%)]	Loss: 0.000000


NameError: ignored